# working notebook

### Utility Functions:
- load_bitcoin_edge_data(filename)
- user_activity_dataframe(bitcoin_df) <-- uses user_stats(bitcoin_df)
- build_graph(bitcoin_df, user_lst=[], rating_type='all', maxdate='2016-01-24')
#### visualization functions:
- plot_timeline(bitcoin_df, title)
- add_user_to_graph(existing_graph, new_user, bitcoin_df)

### EDA Learnings:
alpha network does not have time level timestamp so cannot do velocity or bot analysis
or sort chronologically within a day

### Interesting Fraud Examples:
otc_user = '2680'

In [1]:
import pandas as pd
import numpy as np
import datetime
import networkx as nx

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import sys
sys.path.insert(0, '../src')
import helpers as h
import visualizations as v

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [2]:
# Load and preprocess data
otc_df = h.load_bitcoin_edge_data('../data/soc-sign-bitcoinotc.csv.gz')
alpha_df = h.load_bitcoin_edge_data('../data/soc-sign-bitcoinalpha.csv.gz')
alpha_users = h.user_activity_dataframe(alpha_df)
otc_users = h.user_activity_dataframe(otc_df)

In [3]:
# Load graph objects
_ , alpha_G = h.build_graph(alpha_df)
_ , alpha_pos_G = h.build_graph(alpha_df, rating_type='pos')
_ , otc_G = h.build_graph(otc_df)
_ , otc_pos_G = h.build_graph(otc_df, rating_type='pos')

In [ ]:
v.plot_timeline(alpha_df, 'Alpha Bit Coin Ratings Activity')

In [ ]:
v.plot_timeline(otc_df, 'OTC Bit Coin Ratings Activity')

In [ ]:
alpha_GU = alpha_G.to_undirected()
nx.number_of_cliques(alpha_GU, nodes=7604)

In [ ]:
## Run the triadic census
census = nx.triadic_census(alpha_G)
census

In [ ]:
# triad generator
triads = all_triads(alpha_G)

In [ ]:
node_census = nx.triads_by_type(alpha_G)

In [ ]:
keys=node_census.values()[1].keys() 

## Generate a table header
print('| Node |', ' | '.join(keys))
for k in node_census.keys(): 
     print('|', k, '|',' | '.join([str(v) for v in node_census[k].values()]))


In [ ]:
node_census

In [ ]:
len(census)

In [ ]:
# Connected_component_subgraphs() returns a list of components,
# sorted largest to smallest
components=nx.connected_component_subgraphs(alpha_G)
# pick the first and largest component
cc = components[0]

In [ ]:
from networkx.algorithms import community

communities_generator = community.girvan_newman(alpha_G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)

sorted(map(sorted, top_level_communities))



The Girvan–Newman algorithm detects communities by progressively removing edges from the original graph. The algorithm removes the “most valuable” edge, traditionally the edge with the highest betweenness centrality, at each step. As the graph breaks down into pieces, the tightly knit community structure is exposed and the result can be depicted as a dendrogram.

In [ ]:
import networkx.generators.small

g = networkx.generators.small.krackhardt_kite_graph()

g.adjacency_list()

In [ ]:
otc_users[otc_users['BotActivity']==True].sort_values('TimeActive')

# Need to create visualization to Define Bot acitivity
In Alpha network only bot activity is with zero time delay - all same day raters
In OTC network only 18 users have zero time delay, however, XXX
users have delay under 1? minute (what is delay )

need 3 eda plots of same day rater distribution - day by hour, hour by minute, minute by second

In [ ]:
print(f"Alpha Bot users: {alpha_users['BotActivity'].sum()}")
print(f"OTC Bot users: {otc_users['BotActivity'].sum()}")